Это краткий экскурс в синхронизацию ЭЭГ. Учебный блок будет делиться на структуру общего пайплайна обработки/общий принцип обработки/введение в LSL и подключение serial port / захват триггеров / запись триггеров в csv файл/синхронизация с визуализацией на matplotlib/MATLAB/интеграцию с MNE 

Перед запуском кодов убедитесь, что у вас установлены все необходимые библиотки (psychopy,MNE,MATLAB)

![котик](https://i.pinimg.com/736x/1f/b9/b6/1fb9b636832ca3e90824406d571c1983.jpg)

## Общий принцип составления пайплайнов (пособие по издевательствам над еврейскими детьми)

Trial (проба) — это один повторяющийся элемент эксперимента: например, показ картинки, ожидание ответа, запись результата. Обычно эксперимент состоит из многих однотипных проб, которые отличаются параметрами (стимул, правильный ответ, время показа). В Coder мы чаще всего используем цикл for для перебора этих проб.



In [ ]:
from psychopy import visual, core, event

# Создаём окно
win = visual.Window(size=(800, 600), color='black', units='pix')

# Список слов для показа
words = ['яблоко', 'книга', 'солнце', 'море', 'компьютер'] #Создание словаря из возможных значений слов

# Инструкция
instruction = visual.TextStim(win, text='Сейчас будут появляться слова.\nПросто смотрите на экран.', color='white') #инструкции
instruction.draw()
win.flip()
core.wait(2)

# Цикл по словам
for word in words:   #проходим по каждому элементу в списке(Смотри выше:) 
    # Создаём стимул с текущим словом
    stim = visual.TextStim(win, text=word, color='white', height=40)
    stim.draw()
    win.flip()          # показываем слово
    core.wait(2)        # ждём 2 секунды
    
    # Очищаем экран
    win.flip()
    core.wait(0.5)      # межстимульный интервал

# Сообщение об окончании
thanks = visual.TextStim(win, text='Спасибо за участие!', color='white')
thanks.draw()
win.flip()
core.wait(2)

win.close() #выход из команды
core.quit()

Управление временем осуществляется через логические флаги и циклы while с проверкой времени. Например:


In [ ]:
trial_clock = core.Clock()
stim_presented = False
while trial_clock.getTime() < trial_duration:
    if not stim_presented and trial_clock.getTime() >= stim_onset:
        stimulus.draw()
        win.flip()
        stim_presented = True
        send_trigger(stim_trigger_code)  # отправка триггера
    # проверка ответа
    keys = event.getKeys(keyList=['a','l'])
    if keys:
        response = keys[0]
        rt = trial_clock.getTime()
        # запись ответа
        break
core.wait(0.5)  # межпробный интервал

Для корректного совмещения поведенческих данных и ЭЭГ необходимо передавать маркеры (триггеры) в момент ключевых событий. Самые распространённые способы:

LSL (Lab Streaming Layer) — создаётся outlet, который отправляет строковые или числовые метки. В ЭЭГ-системе (например, LabRecorder) эти метки записываются вместе с сигналом.

Параллельный порт — отправка байтов на LPT-порт, если ЭЭГ-усилитель принимает такие сигналы (требуется драйвер и наличие порта).

Важно: момент отправки триггера должен быть привязан к точному времени отображения стимула. Используйте win.flip() для переключения кадров и сразу после него отправляйте триггер. Время, возвращаемое win.flip(), можно использовать как временную метку события и сохранить в данных.


## логика работы циклов trial

Внутри основного цикла по пробам часто возникают ситуации, требующие условной логики:

Разные типы проб (например, экспериментальные и контрольные) — можно проверять поле условия (if trial['type'] == 'experimental':) и соответствующим образом настраивать стимулы.

Обработка ответов — если ответ правильный, показывать обратную связь, иначе — другой текст.

Ранний выход из пробы — если испытуемый ответил до окончания предъявления, можно прервать показ и перейти к следующей пробе.

Управление временем осуществляется через логические флаги и циклы while с проверкой времени. Например:

In [1]:
trial_clock = core.Clock()
stim_presented = False
while trial_clock.getTime() < trial_duration:
    if not stim_presented and trial_clock.getTime() >= stim_onset:
        stimulus.draw()
        win.flip()
        stim_presented = True
        send_trigger(stim_trigger_code)  # отправка триггера
    # проверка ответа
    keys = event.getKeys(keyList=['a','l'])
    if keys:
        response = keys[0]
        rt = trial_clock.getTime()
        # запись ответа
        break
core.wait(0.5)  # межпробный интервал

NameError: name 'core' is not defined

## Основная схема построения trial

Подготовка пробы

Загрузка параметров текущего условия (тип стимула, длительность, ожидаемый ответ, код триггера и т.д.).

Настройка объектов стимулов (текст, изображение, фигуры) в соответствии с условием.

Предъявление фиксации (опционально)

Отображение фиксационного креста или пустого экрана для подготовки испытуемого.

Фиксация точного времени начала фиксации (через win.flip()).

Предъявление стимула

Вывод целевого стимула на экран.

Фиксация времени предъявления (снова win.flip()).

Отправка синхронизирующего триггера в систему сбора ЭЭГ (сразу после win.flip()).

Сбор ответа (если требуется)

Ожидание реакции испытуемого в течение заданного окна.

Измерение времени реакции (с помощью отдельного таймера).

Возможно, преждевременное завершение показа стимула при ответе.

Межпробный интервал (ITI)

Пустой экран или фиксация между пробами.

Обычно фиксированной или варьируемой длительности.

Сохранение данных пробы

Запись всех параметров и временных меток в структуру данных (список, словарь) для последующего экспорта.

Такой подход обеспечивает точный контроль времени и синхронизацию с внешним оборудованием.



### Как работает Clock()
В психофизических экспериментах критически важна точность временных интервалов: время реакции (RT), 
длительность предъявления стимулов, межстимульные интервалы (ISI), синхронизация с оборудованием (ЭЭГ, айтрекер).
 PsychoPy предоставляет несколько классов для работы со временем, которые позволяют измерять интервалы с высокой точностью (до миллисекунд).
 

In [2]:

from psychopy import core
clock = core.Clock()          # запускается автоматически
t = clock.getTime()           # время в секундах от создания
clock.reset()                  # сброс в 0


ModuleNotFoundError: No module named 'psychopy'

### Болванка цикла с использованием Clock()

В  библиотеке класс AnimatedStim уже использует Clock для анимаций. Конструктор принимает необязательный параметр clock

In [ ]:
def __init__(self, stim, clock=None):
    self.stim = stim
    self.clock = clock or core.Clock()

## Примеры синхронизации Clock с моей библиотекой

показать фиксационный крест, затем цель, замерить время нажатия клавиши

In [ ]:
from psychopy import visual, core, event
from stimuls import FixationCross, ShapeStim   # ваши классы

win = visual.Window(size=(800,600), color='black', units='norm')

# Фиксация
fix = FixationCross(win, line_length=0.1, line_width=0.01, color='white')
fix.show()
win.flip()
core.wait(1.0)

# Цель
fix.hide()
target = ShapeStim(win, shape='circle', radius=0.1, color='red')
target.show()
win.flip()

# Запускаем таймер RT
rt_clock = core.Clock()
keys = event.waitKeys(keyList=['space'], timeStamped=rt_clock)

if keys:
    rt = keys[0][1]   # время нажатия от момента запуска rt_clock
    print(f"RT = {rt:.3f} с")
else:
    print("Нет ответа")

Анимация с синхронизацией через один Clock
Допустим, мы хотим одновременно пульсировать круг и двигать квадрат, используя один таймер для обеих анимаций, чтобы они были согласованы во времени.

In [ ]:
from your_library import ShapeStim, AnimatedStim

win = visual.Window(...)

circle = ShapeStim(win, shape='circle', radius=0.1, color='blue', pos=(-0.3,0))
square = ShapeStim(win, shape='square', radius=0.1, color='green', pos=(0.3,0))
circle.show()
square.show()

# Создаём один общий таймер
common_clock = core.Clock()

anim_circle = AnimatedStim(circle, clock=common_clock)
anim_square = AnimatedStim(square, clock=common_clock)

# Запускаем анимации на 3 секунды
timer = core.Clock()
while timer.getTime() < 3:
    # Генераторы анимаций используют один и тот же common_clock,
    # поэтому они будут синхронны.
    next(anim_circle.pulse(freq=2, min_size=0.8, max_size=1.2))
    next(anim_square.move(speed_x=0.2))  # движение по горизонтали
    win.flip()

Эксперимент с фиксацией триггера 
что почитать



In [ ]:
from psychopy import visual, core, event
from your_library import FixationCross, ShapeStim
import random

win = visual.Window(size=(800,600), color='gray', units='norm', fullscr=False)

# Подготовка
fix = FixationCross(win, line_length=0.08, line_width=0.01, color='black')
target = ShapeStim(win, shape='circle', radius=0.1, color='red')

# Трэкинг времени
rt_clock = core.Clock()
n_trials = 5

for trial in range(n_trials):
    # Фиксация (случайная длительность 0.5–1.5 с)
    fix.show()
    win.flip()
    core.wait(random.uniform(0.5, 1.5))
    
    # Цель
    fix.hide()
    target.show()
    win.flip()
    
    # Измерение RT
    rt_clock.reset()
    keys = event.waitKeys(keyList=['space'], maxWait=2.0, timeStamped=rt_clock)
    
    if keys:
        rt = keys[0][1]
        print(f"Trial {trial+1}: RT = {rt:.3f} с")
    else:
        print(f"Trial {trial+1}: пропуск")
    
    target.hide()
    win.flip()
    core.wait(0.5)  # ISI

win.close()
core.quit()

5. Важные советы и подводные камни
Всегда обновляйте экран после изменения стимулов. win.flip() не только отображает, но и переключает буферы, фиксируя точное время отображения.

Используйте timeStamped в event.waitKeys(), чтобы получить точное время нажатия относительно переданного таймера. Это лучше, чем брать разницу getTime() после выхода из функции.

Не полагайтесь на core.wait() для точных интервалов, если рядом происходят другие операции. Для жёстких временных рамок используйте StaticPeriod.

Генераторы анимации (yield) должны вызываться на каждом кадре. Пропуск кадра приведёт к сбою временной динамики.

При создании StaticPeriod нужно указать screenRefresh — период обновления монитора. Это позволяет классу точно синхронизироваться с вертикальной развёрткой.

Общий таймер для нескольких анимаций удобен, но убедитесь, что все они используют один и тот же экземпляр Clock. Если каждая анимация создаст свой таймер, они будут рассинхронизированы.

В вашем классе AnimatedStim метод drift работает только для стимулов с атрибутом phase (например, GratingStim). Для обычных фигур этот метод не подойдёт.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from psychopy import visual, core, event

# ===== НАСТРОЙКА =====
win = visual.Window([800, 600], color='black', units='pix')
instruction = visual.TextStim(win, text='Нажми любую клавишу как можно быстрее', color='white', height=30)
message_done = visual.TextStim(win, text='Молодец!', color='green', height=40, pos=(0, 0))

# ===== ПРОБА =====
# 1. Подготовка: ничего особенного, просто выводим инструкцию
instruction.draw()
flip_time = win.flip()          # время начала пробы

# 2. Сбор ответа: ждём нажатия, измеряем время реакции
rt_clock = core.Clock()         # запускаем таймер для реакции
keys = []
while not keys:                  # ждём хотя бы одно нажатие
    keys = event.getKeys()
    # Проверка нажатия ESC для выхода
    if 'escape' in keys:
        core.quit()
    core.wait(0.001)             # небольшая пауза, чтобы не грузить CPU

# 3. Регистрация ответа
rt = rt_clock.getTime()          # время реакции (от момента flip_time?)
# Важно: rt_clock был запущен сразу после flip, поэтому rt — это время от начала показа инструкции
# Но точнее: мы запустили rt_clock сразу после win.flip(), значит rt включает и время отрисовки, но это обычно приемлемо.

# 4. Обратная связь (опционально)
message_done.draw()
win.flip()
core.wait(1)

# 5. Сохранение данных (просто вывод в консоль)
print(f'Нажата клавиша: {keys[0]}')
print(f'Время реакции: {rt:.3f} с')

# ===== ЗАВЕРШЕНИЕ =====
win.close()
core.quit()

Давайте посмотрим, как работу внутри циклов можно интегрировать с ООП болванкой для объектов, которую я написал. формат ответов будет в форме 
1. Задание, все задания будут основаны на болванке , приведенной сверху. Если вы тупите, то можете порыться в ссылках для каждого задания. Попробуйте сделать его самостоятельно
2. Если у вас начинает ступор. Записывайте, какую именно часть вы не понимаете, например, как работает цикл? как работать со временем, как прописать логику анимации или как распложит объект. Разберем



## Задание 1

Написать скрипт, где круг движется горизонтально по оси x

![котик](https://i.pinimg.com/736x/1f/b9/b6/1fb9b636832ca3e90824406d571c1983.jpg)

## Обработка и синхрон с ЭЭГ

При проведении экспериментов с регистрацией ЭЭГ крайне важно точно синхронизировать временные метки предъявляемых стимулов и реакций испытуемого с непрерывным потоком данных ЭЭГ. Один из современных и гибких способов добиться этого — использование Lab Streaming Layer (LSL). LSL позволяет передавать данные (в том числе маркеры событий) в реальном времени по локальной сети, а также записывать их в единый файл (например, XDF) для последующего анализа.

Для большей точности привязки к моменту отображения стимула можно использовать win.flip() – он возвращает время фактической смены кадра. Это время и передаётся в LSL.

Вместо core.wait() мы используем цикл while, который проверяет текущее время и позволяет обрабатывать события (например, нажатие клавиш) без блокировки. Такой подход обеспечивает точное соблюдение длительностей без простоя CPU.



### Создание LSL-выхода (outlet) для маркеров
В начале эксперимента создайте поток для отправки строковых маркеров. Обычно это делается один раз.

Отправка маркеров в цикле trials
Типичная структура эксперимента включает цикл по пробным условиям. В каждой пробе мы отправляем маркер в момент начала стимула и, возможно, в момент ответа.

В этом разделе мы разберём, как с помощью библиотеки pylsl организовать обмен данными между PsychoPy и другими программами (например, системой записи ЭЭГ) в реальном времени. Основные понятия, которые нужно освоить – outlet (выходной поток) и inlet (входной поток).

5.1. Что такое LSL и зачем нужны outlet и inlet?
LSL (Lab Streaming Layer) – это сетевая система для передачи потоков данных с единой временной меткой. 
Представьте, что ваш эксперимент – это радиостанция, которая вещает маркеры событий (начало стимула, ответ испытуемого).
 А программа записи ЭЭГ – это приёмник, который ловит эти сигналы и записывает их вместе с ЭЭГ. Чтобы радиостанция могла вещать, ей нужен передатчик (outlet). Чтобы приёмник мог ловить сигнал, ему нужен приёмник (inlet).

В типичном эксперименте PsychoPy выступает в роли «вещателя»: он отправляет маркеры через outlet. Система сбора данных (например, LabRecorder) или программа онлайн-визуализации выступают в роли «приёмников»: они имеют inlet, подписанный на нужный поток.

5.2. Outlet – создаём поток маркеров
Outlet создаётся один раз в начале эксперимента. Для этого нужно описать поток: его имя, тип данных, количество каналов и т.д. Маркеры обычно представляют собой строки (например, "stim_A", "response"), поэтому мы создаём поток с одним строковым каналом.

In [ ]:
from pylsl import StreamInfo, StreamOutlet

# Создаём информацию о потоке маркеров
info = StreamInfo(name='MyMarkers',          # имя потока
                  type='Markers',             # тип потока
                  channel_count=1,             # один канал (строка)
                  nominal_srate=0,              # нерегулярный поток (0)
                  channel_format='string',      # строковый формат
                  source_id='my_exp_markers')   # уникальный ID

# Создаём outlet
outlet = StreamOutlet(info)

После этого можно в любой момент отправить маркер методом push_sample()

In [ ]:
import pylsl
marker_text = "stimulus_onset"
timestamp = pylsl.local_clock()   # берём текущее время по часам LSL
outlet.push_sample([marker_text], timestamp)

## Пример эксперимента. Попробуйте разбить код на логический блоки и расписать для себя схему того, как работает код

In [ ]:
from psychopy import core, visual, event
from pylsl import StreamInfo, StreamOutlet
import random

# --- Создание LSL выхода для маркеров ---
# Описание потока: название "PsychoPy_Events", тип "Markers", 1 канал (целое число), частота нерегулярная
info = StreamInfo('PsychoPy_Events', 'Markers', 1, 0, 'int32', 'myid123')
outlet = StreamOutlet(info)

# --- Окно эксперимента ---
win = visual.Window([800,600], color='black')
text_stim = visual.TextStim(win, text='+', color='white')

# --- Функция отправки маркера с временем события ---
def send_marker(marker_value):
    # Время события (например, момент смены кадра) берём по системным часам PsychoPy
    timestamp = core.getTime()   # или win.timeOnFlip, если нужно точнее привязать к монитору
    outlet.push_sample([marker_value], timestamp)
    print(f"Sent marker {marker_value} at {timestamp:.3f}")

# --- Основной цикл trials ---
n_trials = 10
for trial in range(n_trials):
    # Фиксационный крест (предстимульный период)
    text_stim.text = '+'
    text_stim.draw()
    flip_time = win.flip()           # время смены кадра
    send_marker(1)                    # маркер начала фиксации

    # Ждём 1 секунду (неблокирующее ожидание – используем проверку времени)
    wait_until = flip_time + 1.0
    while core.getTime() < wait_until:
        # Здесь можно проверять нажатия клавиш, обновлять другие стимулы и т.д.
        if event.getKeys(['escape']):
            win.close()
            core.quit()
        pass

    # Стимул (например, изображение или текст)
    stim_value = random.choice([2, 3])   # два типа стимулов
    text_stim.text = str(stim_value)
    text_stim.draw()
    flip_time = win.flip()               # точное время появления стимула
    send_marker(stim_value)               # маркер стимула

    # Межстимульный интервал (ITI)
    wait_until = flip_time + random.uniform(1.5, 2.5)
    while core.getTime() < wait_until:
        if event.getKeys(['escape']):
            win.close()
            core.quit()

win.close()
core.quit()

Еще пример кода(С логической разбивкой)

In [ ]:
from psychopy import visual, event, core
from pylsl import StreamInfo, StreamOutlet, local_clock
import random

Импорт библиотек, ну тут база

In [ ]:
# Создаём описание потока маркеров
info = StreamInfo(
    name='MyExperimentMarkers',      # Имя потока, будет видно в LabRecorder
    type='Markers',                  # Тип потока (стандарт для событий)
    channel_count=1,                  # Один канал (одна строка)
    nominal_srate=0,                   # 0 = нерегулярный поток
    channel_format='string',            # Формат данных - строка
    source_id='visual_experiment'       # Уникальный идентификатор
)

# Создаём сам outlet
outlet = StreamOutlet(info)
print("LSL outlet для маркеров создан. Запустите LabRecorder для записи.")

Что такое канал простыми словами?
Представьте себе таблицу, где каждая строка – это один момент времени, а столбцы – разные измеряемые величины. В LSL каждый такой момент называется образцом (sample). Каналы – это и есть столбцы. Например:

Для маркеров у нас один столбец – текст события.

Для ЭЭГ – много столбцов (например, Fz, Cz, Pz), в каждом – напряжение в этот момент.

Для айтрекера – два столбца: координата X и координата Y.

Когда мы отправляем данные через outlet.push_sample(), мы передаём список, длина которого равна channel_count. Для маркеров это список из одного элемента: [маркер].

Регулярные и нерегулярные потоки
Регулярный поток (nominal_srate > 0) – это поток, в котором данные поступают с постоянной частотой. Например, ЭЭГ-усилитель передаёт значения каждые 2 миллисекунды (500 Гц). Принимающая сторона знает, что образцы следуют строго через равные промежутки времени, и может использовать это для обработки.

Нерегулярный поток (nominal_srate = 0) – данные приходят в случайные моменты времени. Маркеры событий – классический пример: мы отправляем их только тогда, когда что-то происходит (начало стимула, нажатие клавиши). Интервал между маркерами может быть любым. Программа-получатель не должна предполагать какой-то фиксированной частоты.

Важно: Даже для нерегулярного потока каждый образец имеет свою временную метку (которую мы передаём в push_sample), так что временная информация не теряется.

In [ ]:
# Создаём окно эксперимента
win = visual.Window(
    size=[1024, 768],
    color='gray',
    units='pix',
    fullscr=False
)

# Стимул фиксации
fixation = visual.TextStim(win, text='+', color='black', height=40)

# Стимулы для двух условий (квадрат и круг)
stim_square = visual.Rect(win, width=200, height=200, fillColor='red')
stim_circle = visual.Circle(win, radius=100, fillColor='blue')

# Список условий (10 проб каждого типа, перемешаны)
conditions = ['square', 'circle'] * 10
random.shuffle(conditions)

# Инструкция
instruction = visual.TextStim(win, text='Нажмите пробел при появлении фигуры.\nНажмите любую клавишу для начала.', color='black')
instruction.draw()
win.flip()
event.waitKeys()  # ждём нажатия для начала

Создаём окно и несколько стимулов: крестик фиксации, красный квадрат, синий круг.

Формируем список проб в случайном порядке.

Показываем инструкцию и ждём нажатия клавиши (используем event.waitKeys() – он блокирующий, но это допустимо на этапе ожидания начала).

### Отправка стартового маркера

In [ ]:
# Отправляем маркер начала эксперимента
start_time = local_clock()
outlet.push_sample(['experiment_start'], start_time)

Цикл проб(Здесь происходит основаная часть эксперимента)

In [ ]:
for cond in conditions:
    # ---- Межпробный интервал (ITI) ----
    # Длительность ITI варьируется от 1.0 до 1.5 с
    iti_duration = random.uniform(1.0, 1.5)
    iti_end = local_clock() + iti_duration

    # Показываем крестик фиксации во время ITI
    fixation.draw()
    win.flip()  # переключаем буфер, крестик на экране

    # Активное ожидание окончания ITI
    while local_clock() < iti_end:
        # Можно проверять нажатия (например, для досрочного выхода)
        if 'escape' in event.getKeys():
            win.close()
            core.quit()
        core.wait(0.001)  # небольшая задержка для снижения нагрузки на процессор

    # ---- Начало пробы ----
    # Выбираем и отображаем стимул в зависимости от условия
    if cond == 'square':
        stim = stim_square
    else:
        stim = stim_circle

    stim.draw()
    flip_time = win.flip()          # время отрисовки в системе PsychoPy (не используем)
    stim_onset = local_clock()       # время сразу после отрисовки по часам LSL

    # Отправляем маркер начала стимула
    marker = f'stim_{cond}_start'
    outlet.push_sample([marker], stim_onset)

    # Длительность стимула – 1 секунда
    stim_duration = 1.0
    response = None
    rt = None

    # Активное ожидание ответа или окончания стимула
    while local_clock() - stim_onset < stim_duration:
        keys = event.getKeys(['space', 'escape'])
        if 'space' in keys:
            response = 'space'
            rt = local_clock() - stim_onset
            # Отправляем маркер ответа
            outlet.push_sample(['response_space'], local_clock())
            break
        if 'escape' in keys:
            win.close()
            core.quit()
        core.wait(0.001)

    # ---- Конец пробы ----
    # Убираем стимул (показываем крестик или пустой экран)
    fixation.draw()
    win.flip()

    # Отправляем маркер конца стимула (опционально)
    outlet.push_sample([f'stim_{cond}_end'], local_clock())

    # Здесь можно сохранить данные ответа в список для дальнейшего анализа
    # (например, accuracy, RT и т.д.)
    print(f"Проба {cond}: ответ={response}, ВР={rt:.3f}")

## Пояснение ключевых моментов:

Активное ожидание вместо core.wait() – используем цикл while с проверкой local_clock(), что позволяет точно контролировать длительность и мгновенно реагировать на нажатия.

Временная метка стимула берётся сразу после win.flip() – это максимально приближено к моменту реального появления стимула на экране.

Маркеры ответа отправляются с временем нажатия (local_clock()) внутри цикла ожидания, что даёт точное время реакции.

Межпробный интервал тоже управляется через активное ожидание, чтобы не пропустить возможные команды выхода.

In [ ]:
# Отправляем маркер конца эксперимента
outlet.push_sample(['experiment_end'], local_clock())

# Показываем сообщение об окончании
end_msg = visual.TextStim(win, text='Эксперимент завершён. Спасибо!', color='black')
end_msg.draw()
win.flip()
core.wait(2)

win.close()
core.quit()

Запись данных через LabRecorder (внешняя программа)
Во время выполнения эксперимента должна быть запущена программа LabRecorder (входит в состав LSL).

Перед стартом эксперимента в LabRecorder нужно выбрать поток MyExperimentMarkers (созданный нами) и поток вашей ЭЭГ-системы.

Нажать "Start" непосредственно перед запуском скрипта PsychoPy (или после отправки маркера experiment_start).

После окончания эксперимента нажать "Stop" – будет создан файл .xdf, содержащий все потоки с единой временной шкалой.

## Данное издание будет дополняться.

!["meow"](https://i.pinimg.com/1200x/d5/04/ae/d504aeac73da930b0ab511e431bae88c.jpg)